This code was first created 20220323 with the intention to post process UAV imagery based on a CORS Base Station 
This is the second iteration of this code, further cleaning it, and trying to make the code more concise. 

The steps of the code are as follows:
1. Import the RTKLIB corrected Rinex drone data. 
2. Import the Timestamp.MRK file from the UAV.
3. Calculate the camera specific positions from these two files.
4. Scrape the EXIF-data from the original photos to append the pitch, roll, and yaw to the location file. 
    This data can then be used to import into Agisoft.

In [2]:
# Import numerical tools
import numpy as np
# Import pyplot for plotting
import matplotlib.pyplot as plt
#Import pandas for reading in and managing data
import pandas as pd
import math
# Magic function to make matplotlib inlineSet the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.; other style specs must come AFTER
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
#Comment the above line and uncomment the line below if svg graphics are not working in your browser.
#%config InlineBackend.figure_formats = {'png', 'retina'}
import os
import glob
import os.path

Set the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.

In [3]:
# Set the filename. Make sure to keep this value in parenthesis, so that the program reads in this filename 
# number as a named file, and not as an integer.
filename = '101_0170'
# Set the date of the flight to search for the working directory for each time this code is run.
date = '20210505'

Right now, I am going to try and set the working directory for the project, so that we can search for the file types necessary for the RTKLIB and timestamp data. 

In [6]:
# Set your working directory
# Hourglass processing work should be in the folder 'HG_2022_PPK_Processing' and should be parsed out by date
# Since each date will have its own folder, the date in the working directory should be the only thing changed in
# this form.
os.chdir('/Users/f67f911/Desktop/GitHub_PPK/Hourglass_Data_PostProcessing/' + date + '/Raw_Files/')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/f67f911/Desktop/GitHub_PPK/Hourglass_Data_PostProcessing/20210310/Raw_Files/'

Step 1: Read in the already corrected RINEX file from the drone. This file should always be saved in your directory as filename_Rinex.csv so that the following code works. This is from the drone capturing positional location throughout the entirety of the flight, so there will be many more rows of data than there are pictures taken during that flight. 

In [23]:
# Since we have set our working directory, and we have specified the file name, we just need to search for the
# file that is followed by Rinex and is a .csv
RTKLIB_record = pd.read_csv(filename + '_Rinex.csv')
# View the head of the data to make sure it has been read in correctly
RTKLIB_record.head()
# Uncomment the following code if you would like to view the length of the file. 
#len(RTKLIB_record)

,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio
0,2208,399470.8,45.83509,-110.932688,2314.4909,1,6,0.0149,0.0066,0.0224,0.0046,0.0076,0.0148,-4.2,527.1
1,2208,399471.0,45.83509,-110.932688,2314.4880,1,6,0.0147,0.0065,0.0220,0.0045,0.0075,0.0145,-4.0,526.5
2,2208,399471.2,45.83509,-110.932688,2314.4906,1,6,0.0144,0.0064,0.0217,0.0044,0.0073,0.0143,-3.8,526.0
3,2208,399471.4,45.83509,-110.932688,2314.4864,1,6,0.0142,0.0062,0.0213,0.0043,0.0072,0.0141,-3.6,525.8
4,2208,399471.6,45.83509,-110.932688,2314.4932,1,6,0.0139,0.0061,0.0210,0.0042,0.0070,0.0138,-3.4,525.7


Step 2: Read in the timestamp data as is from the UAV. 

In [24]:
# Again, since we have a single working directory, read in the timestamp data with the name of the file.
timestamp_record = pd.read_table(filename + '_Timestamp.MRK', header = None)
# The UAV used, a P4 RTK, always collects timestamp files with the data in the same order. Therefore, we can set the 
# column names for the timestamp file read in. 
timestamp_record.columns = ['Photo', 'GPS_Date','% GPST','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m','std_North_m, std_East_m, std_Ele_m','RTK_status_flag']
# View the head of the data to make sure it has been read in correctly. 
timestamp_record.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag
0,1,399477.727242,[2208],"-9,N","-4,E","194,V","45.83506059,Lat","-110.93269097,Lon","2317.334,Ellh","0.012661, 0.021600, 0.029364","50,Q"
1,2,399481.901610,[2208],"10,N","-16,E","193,V","45.83513023,Lat","-110.93275864,Lon","2307.852,Ellh","0.012964, 0.022205, 0.029983","50,Q"
2,3,399485.464902,[2208],"15,N","-21,E","193,V","45.83519589,Lat","-110.93282491,Lon","2298.959,Ellh","0.012870, 0.021786, 0.030213","50,Q"
3,4,399486.986303,[2208],"16,N","-21,E","192,V","45.83520890,Lat","-110.93283758,Lon","2297.254,Ellh","0.013342, 0.022087, 0.030188","50,Q"
4,5,399491.646347,[2208],"0,N","3,E","194,V","45.83522713,Lat","-110.93285622,Lon","2309.734,Ellh","0.013127, 0.021502, 0.030736","50,Q"


Note status flag values - 0: no positioning; 16: single-point positioning mode; 34:RTK floating solution; 50: RTK fixed solution. When flag of a photo is not equal to 50, it is recommended that you should not use that image in further processing.

In [10]:
# Create a holding record for the timestamp data in case the dataframe is messed up
timestamp = timestamp_record

Clean the timestamp file to convert non-numeric text in columns to numeric

When looking at our columns, we can see that there are many numbers followed by letters. We need to get rid of those numbers in order to continue with our analysis. This code can definitely be cleaned up. For example, right now it works because all of the flight logs I am using take place in the United States, meaning that the letters following the timestamp data will all be the same. However, this is not necessarily the case in other areas.

In the future, I could try to remove anything from the columns that is not an integer, so that letters and commas are removed. However, right now, this works fine. 

In [11]:
timestamp['Northing_diff_mm'] = timestamp['Northing_diff_mm'].str.replace(',N', '')
timestamp['Easting_diff_mm'] = timestamp['Easting_diff_mm'].str.replace(',E', '')
timestamp['Elevation_diff_mm'] = timestamp['Elevation_diff_mm'].str.replace(',V', '')
timestamp['Lat'] = timestamp['Lat'].str.replace(',Lat','')
timestamp['Lon'] = timestamp['Lon'].str.replace(',Lon','')
timestamp['Height_m'] = timestamp['Height_m'].str.replace(',Ellh','')
timestamp['RTK_status_flag'] = timestamp['RTK_status_flag'].str.replace(',Q','')

In [12]:
# Look at the datatypes for the timestamps. The way the csv is read in, the column data is not necessarily in the 
# correct format. We can check what data type each of the columns is, and change as needed.
timestamp.dtypes

Photo                                   int64
GPS_Date                              float64
% GPST                                 object
Northing_diff_mm                       object
Easting_diff_mm                        object
Elevation_diff_mm                      object
Lat                                    object
Lon                                    object
Height_m                               object
std_North_m, std_East_m, std_Ele_m     object
RTK_status_flag                        object
dtype: object

In [13]:
# Let's change those datatypes so we can actually do calculations on the column values
timestamp['Northing_diff_mm'] = timestamp['Northing_diff_mm'].astype(int)
timestamp['Easting_diff_mm'] = timestamp['Easting_diff_mm'].astype(int)
timestamp['Elevation_diff_mm'] = timestamp['Elevation_diff_mm'].astype(int)
timestamp['Lat'] = timestamp['Lat'].astype(float)
timestamp['Lon'] = timestamp['Lon'].astype(float)
timestamp['Height_m'] = timestamp['Height_m'].astype(float)
timestamp.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag
0,1,399477.727242,[2208],-9,-4,194,45.835061,-110.932691,2317.334,"0.012661, 0.021600, 0.029364",50
1,2,399481.901610,[2208],10,-16,193,45.835130,-110.932759,2307.852,"0.012964, 0.022205, 0.029983",50
2,3,399485.464902,[2208],15,-21,193,45.835196,-110.932825,2298.959,"0.012870, 0.021786, 0.030213",50
3,4,399486.986303,[2208],16,-21,192,45.835209,-110.932838,2297.254,"0.013342, 0.022087, 0.030188",50
4,5,399491.646347,[2208],0,3,194,45.835227,-110.932856,2309.734,"0.013127, 0.021502, 0.030736",50


Calculate camera specific positions. Step one is to create a calculations spreadsheet.

In [1]:
# In the future, I think I can drop the Closest_Loc_ID, since the merging way that I am doing the data will 
# mess up this ID, and therefore will negate the need to have these columns.
calc = pd.DataFrame(columns = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Closest_Loc_ID',
                    'Timestamp_of_Closest','Closest_Lat','Closest_Lon','Closest_El','2nd_Closest_Loc_ID',
                    'Timestamp_of_2nd_Closest','2nd_closest_Lat','2nd_Closest_Lon','2nd_Closest_El',
                    'Percent_diff_between_timestamps','Interpolated_Lat','Interpolated_Lon','Interpolated_El',
                    'Lat_Diff_deg','Lon_Diff_deg','El_diff_m','New_Lat','New_Lon','New_El']).astype(int)

NameError: name 'pd' is not defined

Step 3.1: Calculate the values and input into the calc dataframe. This output will include a new latitude, longitude, and elevation.

In [15]:
calc['Northing_diff_mm'] = timestamp['Northing_diff_mm']
calc['Easting_diff_mm'] = timestamp['Easting_diff_mm']
calc['Elevation_diff_mm'] = timestamp['Elevation_diff_mm']
# Read in the data to make sure these columns have populated
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,-9,-4,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,-16,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,-21,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,-21,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,3,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Step 3: Convert the latitude difference into degrees.
    

In [25]:
# First, we need to create constant values with numbers used in further calculations as conversion factors
# 1 degree latitude in meters
deg_lat_m = 111111
# The latitude used
Lat_used =  45.83505277 
# 1 degree longitude in meters
deg_lon_m = 77414

In [17]:
# We need to make sure the northing and easting differences are changed to a type int, so that wer can do our
# further calculations on the values.
calc['Northing_diff_mm'] = calc['Northing_diff_mm'].astype(int)
calc['Easting_diff_mm'] = calc['Easting_diff_mm'].astype(int)
calc['Elevation_diff_mm'] = calc['Elevation_diff_mm'].astype(int)

In [18]:
# This code calculates the latitude difference in degrees for this dataset. 
calc['Lat_Diff_deg'] = calc['Northing_diff_mm']/1000/deg_lat_m
# The longitude difference in degrees
calc['Lon_Diff_deg'] = calc['Easting_diff_mm']/1000/ deg_lon_m
# The elevation difference in meters
calc['El_diff_m'] = calc['Elevation_diff_mm']/1000
# Call the head of the dataframe to make sure these calculations were done correctly
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,-9,-4,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-8.100008e-08,-5.167024e-08,0.194,NaN,NaN,NaN
1,10,-16,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.000009e-08,-2.066810e-07,0.193,NaN,NaN,NaN
2,15,-21,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.350001e-07,-2.712688e-07,0.193,NaN,NaN,NaN
3,16,-21,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.440001e-07,-2.712688e-07,0.192,NaN,NaN,NaN
4,0,3,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000e+00,3.875268e-08,0.194,NaN,NaN,NaN


Calculate the closest latitude. To do this, we first need to calculate the timestamp that is closest.

In [19]:
# First lets add a column in the RTKLIB data to populate out the timestamps.
# Set a record in case we mess up this data
RTKLIB = RTKLIB_record

Please note that this code is still in process!! This code does a good job of populating the new lat, lon, and elevation data based on the observed differences, but work is still needed! That is why there are still columns that have not populated, columns that will create calculations that are even more accurate for the UAV! For now, this code may suffice, but I would like to improve it in the future. 

In [20]:
calc['New_Lat'] = timestamp['Lat'] + calc['Lat_Diff_deg']
calc['New_Lon'] = timestamp['Lon'] + calc['Lon_Diff_deg']
calc['New_El'] = timestamp['Height_m'] + calc['El_diff_m']
calc

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,-9,-4,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-8.100008e-08,-5.167024e-08,0.194,45.835061,-110.932691,2317.528
1,10,-16,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.000009e-08,-2.066810e-07,0.193,45.835130,-110.932759,2308.045
2,15,-21,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.350001e-07,-2.712688e-07,0.193,45.835196,-110.932825,2299.152
3,16,-21,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.440001e-07,-2.712688e-07,0.192,45.835209,-110.932838,2297.446
4,0,3,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000e+00,3.875268e-08,0.194,45.835227,-110.932856,2309.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,19,0,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.710002e-07,0.000000e+00,0.194,45.832769,-110.937440,2542.612
214,-9,27,191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-8.100008e-08,3.487741e-07,0.191,45.832794,-110.937464,2541.263
215,19,8,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.710002e-07,1.033405e-07,0.194,45.832887,-110.937556,2544.813
216,17,7,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.530002e-07,9.042292e-08,0.194,45.832975,-110.937645,2548.043


Now, we need to call the original photo EXIF information to adjust pitch, roll, and yaw. 

In [32]:
# Install the exiftool package to scape the exif data from the images.
pip install PyExifTool

     |████████████████████████████████| 49 kB 467 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import exiftool

In [35]:
pics = filename + '.jpg'


'101_0340.jpg'

Step 5: Create a file to export for image processing.

In [21]:
# Create a new file dataframe structure for export
export = pd.DataFrame(columns = ['New_Lat', 'New_Lon', 'New_El'])
export

,New_Lat,New_Lon,New_El


In [41]:
# Populate the columns based on the calc dataframe
export['New_Lat'] = calc['New_Lat']
export['New_Lon'] = calc['New_Lon']
export['New_El'] = calc['New_El']
export

,New_Lat,New_Lon,New_El
0,45.835058,-110.932687,2317.424
1,45.835132,-110.932762,2308.044
2,45.835197,-110.932825,2299.223
3,45.835210,-110.932836,2297.512
4,45.835228,-110.932856,2309.877
...,...,...,...
213,45.832769,-110.937441,2542.605
214,45.832794,-110.937464,2541.278
215,45.832886,-110.937555,2544.796
216,45.832975,-110.937645,2548.001


In [42]:
# Now export the data for further processing!!!
export.to_csv(r'/Users/f67f911/Desktop/HG2022_Data/corrected_gps.csv')